Theano の 補講
=======

担当者　小川 兼司

誤解を恐れず書いている部分もあるので、その点ご了承下さい。


## 私見   Thenoについて

機械学習、Deeplearningのツールは色々ありますが、

* Deeplearningの仕組みを理解しながらコードを書く

* 手っ取り早くGPUを使う

* pythonでコードを書く

という場合はTheanoがベスト


## 他のライブラリとの比較

Caffe, Torch, Chainer　
    
    各層のブロックを積み重ねるという考え方(既成のコード(関数)の積み重ね)

    GPUを使用したままでのカスタマイズは困難（もともとそういう目的で作られてない）

Theano
    
    各層のもとになる関数を自分で書く事ができる(C/C++, GPU用のコードが実行時に生成、コンパイルされる)
    
    Neural Networkの為の仕組みは用意されている
    
    Theano特有の文法を知っている必要がある
    

## GPUについて知っておいて欲しい事

Graphics Processing Unit の略, CPUの外部にあるデバイス

Deeplearning、科学技術などの数値計算で使われるのはほとんどNvidiaの製品 (Tesla, GeForce)

本来の用途は画像処理、たくさんの小さい演算装置の集まり

    SIMD (single instruction multiple data)

    (3 color + transparency) のデータを一まとまりにする (4 unsigned int) => 4 sp float, 2 dp float

    single precisionの最適化されたコードとdouble precisionの最適化されたコードは全く違う。

GPUのメモリはCPUとは全く別のところにある。

CUDAというプラットフォームを使用 (C/C++に近いコーディング。Theanoを使う場合はGPUの仕組み、GPU特有のコーディングについて"それほど"意識しなくてもいい)

とりありず、以下の２つだけは心に留めておいて下さい

* ### GPU は SIMDしか高速化できない

* ### GPUに渡すデータの並び順は高速化をする上で重要


## Theanoを使う上での注意

* ### GPUで高速化できるのは float32(単精度浮動小数) の計算のみ

    これはTheano特有の問題

#### 参照

Theano

    http://deeplearning.net/software/theano/tutorial/

Deeplearning Tutorial

    http://deeplearning.net/tutorial/contents.html

Theano製作者の論文（Theanoを使った研究論文を書く時にはリファレンスに入れる）

    http://deeplearning.net/software/theano/citation.html

誰かのブログ（よく書けてます）

    http://aidiary.hatenablog.com/entry/20150509/1431137590
    

GPU (CUDA)

    http://www.nvidia.co.jp/object/cuda-jp.html


Libraryの比較について

    http://deeplearning4j.org/compare-dl4j-torch7-pylearn.html

    http://fastml.com/torch-vs-theano/


Thenoを使ったライブラリ

    Keras
    https://github.com/fchollet/keras
    
    Lasagne(ラザニア)
    https://lasagne.readthedocs.org/en/latest/
    
    Blocks
    https://github.com/mila-udem/blocks
    
    その他 github
    https://github.com/search?utf8=%E2%9C%93&q=Theano&type=Repositories&ref=searchresults
    

                                                                                                            (cell 1)

## このノートに書いた事（順不同）

* theanoに用意されている関数
    tanh, sigmoid

* theano.function　の使い方

* givens の使い方

* theano.shared と　theano.tensor.scalar, theano.tensor.vectorなどの違い

* 乱数について

* grad, updates　の使い方

* その他の注意点

(cell 1.1)





## Theano にはどんな関数があるか

In [1]:
# cell 2
import theano
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")

dir(theano.tensor)



Using gpu device 0: GeForce 820M


['ARange',
 'AdvancedIncSubtensor',
 'AdvancedIncSubtensor1',
 'AdvancedIndexingError',
 'AdvancedSubtensor',
 'AdvancedSubtensor1',
 'Alloc',
 'Apply',
 'AsTensorError',
 'CAReduce',
 'Choose',
 'Constant',
 'Default',
 'Diag',
 'Diagonal',
 'DiffOp',
 'DimShuffle',
 'DisconnectedType',
 'Dot',
 'Elemwise',
 'EmptyConstantError',
 'Eye',
 'Flatten',
 'Generic',
 'IncSubtensor',
 'Join',
 'LoadFromDisk',
 'Lop',
 'MPIRecv',
 'MPIRecvWait',
 'MPISend',
 'MPISendWait',
 'MakeSlice',
 'MaxAndArgmax',
 'Mean',
 'MethodNotDefined',
 'NoneConst',
 'NoneTypeT',
 'Nonzero',
 'NotScalarConstantError',
 'NumpyAutocaster',
 'Op',
 'PermuteRowElements',
 'Rebroadcast',
 'Reshape',
 'Rop',
 'ScalarFromTensor',
 'Shape',
 'ShapeError',
 'SliceConstant',
 'SliceType',
 'SpecifyShape',
 'Split',
 'Subtensor',
 'SubtensorPrinter',
 'Sum',
 'Tensor',
 'TensorConstant',
 'TensorConstantSignature',
 'TensorFromScalar',
 'TensorType',
 'TensorVariable',
 'Tile',
 'Tri',
 'Type',
 'Variable',
 '__builtins__

In [2]:
# cell 3
import theano
dir(theano.tensor.nnet)



['Apply',
 'Conv3D',
 'ConvGrad3D',
 'ConvOp',
 'ConvTransp3D',
 'CrossentropyCategorical1Hot',
 'CrossentropyCategorical1HotGrad',
 'CrossentropySoftmax1HotWithBiasDx',
 'CrossentropySoftmaxArgmax1HotWithBias',
 'DisconnectedType',
 'N',
 'Prepend_scalar_constant_to_each_row',
 'Prepend_scalar_to_each_row',
 'Softmax',
 'SoftmaxGrad',
 'SoftmaxWithBias',
 'T',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 'binary_crossentropy',
 'blas_header_text',
 'blas_header_version',
 'categorical_crossentropy',
 'computeH',
 'computeR',
 'conv',
 'conv2d',
 'conv3D',
 'convGrad3D',
 'convTransp3D',
 'crossentropy_categorical_1hot',
 'crossentropy_categorical_1hot_grad',
 'crossentropy_softmax_1hot',
 'crossentropy_softmax_1hot_with_bias',
 'crossentropy_softmax_1hot_with_bias_dx',
 'crossentropy_softmax_argmax_1hot_with_bias',
 'crossentropy_softmax_max_and_argmax_1hot',
 'crossentropy_softmax_max_and_argmax_1hot_with_bias',
 'crossentropy_to_crossentropy_w

In [3]:
# cell 4
# Frequently used function in NN
print 'tanh' in dir(theano.tensor)
print 'sigmoid' in dir(theano.tensor.nnet)



True
True


## theano.function

In [11]:
# cell 5
import theano
import numpy as np
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")



# You CANNOT DO this.
f = theano.function([],
                    np.ones(3))

f()



TypeError: ('output must be a theano Variable or Out instance (or list of them)', array([ 1.,  1.,  1.]))

In [5]:
# cell 6
import theano
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")

# You CAN DO this.
f = theano.function([],
                    theano.tensor.ones(3))

f()



array([ 1.,  1.,  1.])

In [6]:
# cell 7.1
# Theano scalar, vector cannot have it's value
# It's used to define function
import theano
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")

a = theano.tensor.scalar(dtype='float32')
b = theano.tensor.scalar(dtype='float32')
c = a + b
print a, b, c

tf_add = theano.function([a, b], c)

print tf_add(1, 3)



<TensorType(float32, scalar)> <TensorType(float32, scalar)> Elemwise{add,no_inplace}.0
4.0


In [7]:
# cell 7.2
# The memory are allocated for shared valiable

import theano
import numpy as np
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")

x = theano.shared(np.arange(10, dtype='float32'))
y = theano.shared(np.ones((10,)))

print x.get_value()
print y.get_value()
print x.get_value() + y.get_value()



[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]


In [8]:
# cell 7.3
# givens

import theano
import numpy as np
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")

a = theano.tensor.scalar(dtype='float32')
b = theano.tensor.scalar(dtype='float32')
c = a + b
print a, b, c

d = theano.shared(np.float32(10))
tf_add = theano.function([a], c, givens={b: d})

print tf_add(3)



<TensorType(float32, scalar)> <TensorType(float32, scalar)> Elemwise{add,no_inplace}.0
13.0


In [12]:
# cell 7.4
# grad
import theano
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")

tx = theano.tensor.vector(dtype='float32')
ty = (tx * tx * tx).sum()

grad_y = theano.grad(ty, tx)
tf_grad_y = theano.function([tx], ty)

x = np.ones(1, dtype='float32') * 0.5
print tf_grad_y(x)



0.125


In [10]:
# cell 7.5
# update
tx = theano.tensor.vector(dtype='float32')
s = theano.shared(np.float32(0))

tx = theano.tensor.scalar(dtype='float32')
acumulate = theano.function([tx], updates={s: s + tx})

# sum of 0, 1, 2, .... 10
for i in range(11):
    acumulate(i)
print s.get_value()



55.0


In [14]:
# cell 8
# numpy binomial

import numpy as np

rng = np.random.RandomState()
rng.binomial(n=1, p=0.5, size=(10,))



array([1, 0, 1, 0, 1, 0, 0, 1, 1, 0])

### 関数定義時にされる事、関数実行時にされる事

In [13]:
# cell 9.1
import theano
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")

srng = theano.tensor.shared_randomstreams.RandomStreams(123)
binom = srng.binomial(n=1, p=0.5, size=(10,))

tf_binomial = theano.function([], binom)

for i in range(10):
    print tf_binomial()



[1 1 0 1 1 0 0 1 0 1]
[1 0 1 0 1 1 1 1 0 1]
[1 0 0 0 1 1 1 1 1 1]
[0 1 1 0 0 0 0 0 0 0]
[0 1 0 1 1 1 0 0 0 1]
[0 1 1 1 0 0 1 0 1 0]
[1 1 1 1 0 1 1 0 1 1]
[1 0 1 0 0 0 0 0 0 1]
[0 1 1 0 0 1 0 1 1 1]
[0 0 0 1 1 0 1 0 1 1]


In [15]:
# cell 9.2
import theano
import numpy as np
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")

rng = np.random.RandomState()
binom = theano.shared(rng.binomial(n=1, p=0.5, size=(10,)))
tf_binomial = theano.function([], binom)

for i in range(10):
    print tf_binomial()



[1 0 0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 1 0 0 0]


cell 9.1 において、theano.tensor.shared_randomstreams　を使わずに同じような結果を出すにはどうすればいいでしょうか？

In [16]:
# cell 9.3
#import theano
#import numpy as np
#rng = np.random.RandomState()
#binom = theano.shared(rng.binomial(n=1, p=0.5, size=(10*10,)))
#i = theano.tensor.iscalar()
#x = theano.tensor.vector('float32')
#tf_binomial = theano.function(inputs=[i], outputs=x, givens={x:binom[i*10:(i+1)*10]})
#
# for i in range(10):
#    print tf_binomial(i)



### データタイプは常に明示的にするべきです。

In [17]:
# cell 10.1
x = theano.tensor.constant(127)
print x.get_scalar_constant_value(), x.dtype

a = x.get_scalar_constant_value()

x = theano.tensor.constant(128)
print x.get_scalar_constant_value(), x.dtype

b = x.get_scalar_constant_value()

print a + a

print b + b



127 int8
128 int16
-2
256


上のセルの３行めはなぜこうなったのでしょうか？　これを127+127=254にするにはどうすればいいでしょうか。

In [18]:
# cell 10.2
#x = theano.tensor.constant(127,dtype='int16')
# print x.get_scalar_constant_value(), x.dtype
#a = x.get_scalar_constant_value()
# print a + a



### 微分不可能な点がある場合

In [19]:
import theano
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu0")

x = theano.tensor.scalar()
y = x * (x>0)
#y = x * (x>=0)
d = theano.grad(y,[x])
fd = theano.function([x],d)
print fd(0.1), fd(0.0), fd(-0.1)

[array(1.0)] [array(0.0)] [array(0.0)]


## NaNが現れるケース

In [20]:
import numpy as np
import theano
#import theano.sandbox.cuda
#theano.sandbox.cuda.use("gpu0")

#x = theano.tensor.scalar(dtype='float64')
x = theano.tensor.scalar(dtype='float32')

y = (1.0/x) * x
#y = (1.0/x)
#y = (1.0/x) - (1.0/x)

d = theano.grad(y,x)

fd = theano.function([x], d)
fy = theano.function([x], y)
print np.float32(1.0e-45), np.float32(1.0e-46)
print fd(0.0), fd(1.0e+310), fd(np.float32(1.0e-38)), fd(np.float32(1.0e-39))
print fy(0.0), fy(1.0e+310), fy(np.float32(1.0e-38)), fy(np.float32(1.0e-39))
#print fd(1.0e-308), fd(1.0e-309), fd(0.0), fd(1.0e+310), fd(1.e-38), fd(1.e-39)
#print fy(1.0e-308), fy(1.0e-309), fy(0.0), fy(1.0e+310), fy(1.e-38), fy(1.e-39)

1.4013e-45 0.0
nan 0.0 0.0 nan
1.0 1.0 1.0 1.0


#### 質問　theanoを使う上で初心者のハマりやすいところや知っとくとためになるコツがあれば教えていただきたいです 

私が知る限りではだいたい、このノートに書いたとおりです。

#### 質問　theano.gradでは、任意の微分可能関数に対して微分は計算されるのでしょうか？

'任意の'と聞かれたら、私は答えを知りません。

たとえば、複素関数で複雑な多価関数になった場合など、私は一つ一つ調べたわけではないので分かり兼ねます。

私の経験では機械学習で使用するほとんどの関数はtheanoで構成できて微分の計算ができます。

数値計算上での有効桁数により制約をうける事もあります。